# Bathymetric Data Visualization
- __Objective:__ Acquire vector or high-definition raster images of equal-depth lines from Meditteranean bathymetry data.
- __Data:__ You can send requests to [this URL](https://ows.emodnet-bathymetry.eu/wcs?service=wcs&version=1.0.0&request=getcoverage&coverage=) to get raster bathymetry data.  This bathymetric data is recorded and published by the European Marine Observation and Data Network (EMODnet) via their Web Coverage Service (WCS; specifically for posting raster data).

In [1]:
# You can find more detail on these libraries by consulting my install script or
# visiting their page on CRAN.
library(XML)
library(ncdf4)
library(sp)
library(terra)
library(downloader)
library(ggplot2)
library(directlabels)
library(rasterVis)
library(raster)

terra 1.7.55

Loading required package: lattice



In [2]:
source("r_tutorial/helpers.r")

In [3]:
xmin <- 22.0
xmax <- 24.5
ymin <- 39.0
ymax <- 41.0

In [4]:
get_bathymetry <- function(xmin = 22.5, xmax = 25, ymin = 37.5, ymax = 40, resx=1, resy=1) {
  name <- "emodnet:mean"
  bounding_box <- paste(xmin, ymin, xmax, ymax, sep = ",")
  url <- paste("https://ows.emodnet-bathymetry.eu/wcs?service=wcs&version=1.0.0&request=getcoverage&coverage=",
               name, "&crs=EPSG:4326&BBOX=", bounding_box,
               "&format=image/tiff&interpolation=nearest&resx=", resx, "&resy=", resy,
               sep = "")
  print(url)
  temp <- paste(name, "img.tiff", sep = "_")
  temp <- tempfile(temp)
  download.file(url, temp, quiet = FALSE, mode = "wb")
  img_raw <- raster(temp)
  img_raw <- -1*img_raw
  img_raw[img_raw < 0] <- 0
  img_raw[img_raw == 0] <- NaN
  return(img_raw)
}

In [5]:
img <- get_bathymetry(xmin, xmax, ymin, ymax, resx = 0.01, resy=0.01)

[1] "https://ows.emodnet-bathymetry.eu/wcs?service=wcs&version=1.0.0&request=getcoverage&coverage=emodnet:mean&crs=EPSG:4326&BBOX=22,39,24.5,41&format=image/tiff&interpolation=nearest&resx=0.01&resy=0.01"


In [6]:
bathy <- as.data.frame(as(img, "SpatialPixelsDataFrame"))

In [7]:
head(bathy)

,emodnet.mean_img,x,y
,<dbl>,<dbl>,<dbl>
1,0.08326514,24.485,40.965
2,4.41793442,24.485,40.955
3,4.65628767,24.495,40.955
4,2.91300297,24.435,40.945
5,2.64227533,24.475,40.945
6,13.21458912,24.485,40.945


In [14]:
thessaloniki_coords <- data.frame(x = c(40.6401), y = c(22.9444))

In [17]:
map <- ggplot(aes(x = x, y = y, z = emodnet.mean_img), data = bathy) +
  geom_raster(data = bathy, aes(fill = emodnet.mean_img)) +
  geom_point(data = thessaloniki_coords, aes(x = x, y = y, fill = "#000000"), inherit.aes = FALSE) +
  scale_fill_gradient(low = "white", high = "darkblue", name = "Depth (m)") +
  coord_quickmap(xlim = range(xmin, xmax), ylim = range(ymin, ymax)) +
  ggtitle("EMODnet Bathymetry") + xlab("Longitude") + ylab("Latitude") +
  theme_bw()
map

Warning message:
“Raster pixels are placed at uneven horizontal intervals and will be shifted
ℹ Consider using `geom_tile()` instead.”


ERROR while rich displaying an object: Error: Discrete value supplied to continuous scale

Traceback:
1. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
2. tryCatchList(expr, classes, parentenv, handlers)
3. tryCatchOne(expr, names, parentenv, handlers[[1L]])
4. doTryCatch(return(expr), name, parentenv, handler)
5. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
6. repr::mime2repr[[mime]](obj)
7. repr_text.default(obj)
8. paste(capture.output(print(ob

In [13]:
goo = data.frame(x = c(40.6401), y = c(22.9444))
goo

x,y
<dbl>,<dbl>
40.6401,22.9444


In [9]:
object_type(c(1,1))

[1] "vector"

In [10]:
foo <- data.frame(x=c(1), y=c(2))
foo

x,y
<dbl>,<dbl>
1,2
